In [1]:
#include <assert.h>
#include <iostream>
#include <string>
#include <fstream>

In [2]:
template<unsigned int dim, typename T>
struct Matrix {
    unsigned int size;
    unsigned int shape[dim];
    T* data = NULL;
    ~Matrix() { delete[] data; }
    void allocate(const unsigned int (&shape_)[dim]);
    //void allocate(unsigned int (&&shape_)[dim]) { allocate(shape_); };
    T& elem(const unsigned int (&multidim_index)[dim]) const;
    //T& elem(unsigned int (&&index)[dim]) const { return elem(index); };
    unsigned int multidim2flat_index(const unsigned int (&multidim_index)[dim]) const;
    void flat2multidim_index(unsigned int flat_index, unsigned int (&multidim_index)[dim]) const;
    void print_elem(unsigned int flat_index) const;
    void print(unsigned int limit = -1) const;
};

template<unsigned int dim, typename T>
void Matrix<dim, T>::allocate(const unsigned int (&shape_)[dim]) {
    size = 1;
    for(unsigned int dim_index = 0; dim_index < dim; ++dim_index) {
        shape[dim_index] = shape_[dim_index];
        size *= shape_[dim_index];
    }
    if(data) { delete[] data; }
    data = new T[size] {0};
}

template<unsigned int dim, typename T>
T& Matrix<dim, T>::elem(const unsigned int (&multidim_index)[dim]) const {
    unsigned int flat_index = multidim2flat_index(multidim_index);
    return data[flat_index];
}

template<unsigned int dim, typename T>
unsigned int Matrix<dim, T>::multidim2flat_index(const unsigned int (&multidim_index)[dim]) const {
    for (unsigned int dim_index; dim_index < dim; ++dim_index)
        assert(multidim_index[dim_index] < shape[dim_index]);
    unsigned int flat_index = 0;
    unsigned int stride = 1;
    for(int dim_index = dim - 1; dim_index >= 0; --dim_index) {
        flat_index += stride * multidim_index[dim_index];
        stride *= shape[dim_index];
    }
    return flat_index;
}

template<unsigned int dim, typename T>
void Matrix<dim, T>::flat2multidim_index(unsigned int flat_index, unsigned int (&multidim_index)[dim]) const {
    for(int dim_index = dim - 1; dim_index >= 0; --dim_index) {
        multidim_index[dim_index] = flat_index % shape[dim_index];
        flat_index /= shape[dim_index];
    }
}

template<unsigned int dim, typename T>
void Matrix<dim, T>::print(unsigned int limit) const {
    // print size, shape and (some) elements of the matrix
    std::cout << "size: " << size << std::endl;
    std::cout << "shape: {";
    for(unsigned int dim_index = 0; dim_index < dim - 1; ++dim_index)
        std::cout << shape[dim_index] << ", ";
    std::cout << shape[dim - 1] << "}" << std::endl;
    if(data) {
        std::cout << "elements:" << std::endl;
        if (size <= limit) {
            if (dim == 2) {
                for (unsigned int i = 0; i < shape[0]; ++i) {
                    for (unsigned int j = 0; j < shape[1]; ++j) {
                        std::cout << elem({i, j}) << "\t";
                    }
                    std::cout << std::endl;
                }
            } else {
                for(unsigned int flat_index = 0; flat_index < size; ++flat_index)
                    print_elem(flat_index);
            }
        }
        else {
            for(unsigned int flat_index = 0; flat_index < (limit + 1) / 2; ++flat_index)
                print_elem(flat_index);
            std::cout << "..." << std::endl;
            for(unsigned int flat_index = size - limit / 2; flat_index < size; ++flat_index)
                print_elem(flat_index);
        }
    }
}
                
template<unsigned int dim, typename T>
void Matrix<dim, T>::print_elem(unsigned int flat_index) const {
    // print multidimensional index and the value
    unsigned int multidim_index[dim];
    flat2multidim_index(flat_index, multidim_index);
    std::cout << flat_index << " [";
    for(unsigned int dim_index = 0; dim_index < dim - 1; ++dim_index)
        std::cout << multidim_index[dim_index] << ", ";
    std::cout << multidim_index[dim - 1] << "] " << +data[flat_index] << std::endl;
}

In [3]:
// Definitions
// strictly for the sake of IO

template<typename T>
struct Data : Matrix<2, T> {
    void read_transposing(std::string &&file_name, char separator = ',');
    void allocate(std::string &file_name, char separator);
    T* row(unsigned int row_index) const;
};

template<typename T>
void Data<T>::read_transposing(std::string &&file_name, char separator) {
    allocate(file_name, separator);
    // populate the matrix with 
    // transposed data from the file
    std::ifstream file(file_name);
    std::string text_line, text_placeholder;
    for(unsigned int j = 0; j < this->shape[1]; ++j) {
        std::getline(file, text_line);
        std::stringstream text_line_stream(text_line);
        for(unsigned i = 0; i < this->shape[0]; ++i) {
            std::getline(text_line_stream, text_placeholder, separator);
            this->elem({i, j}) = std::stod(text_placeholder);
        }
    }
    file.close();
}

template<typename T>
void Data<T>::allocate(std::string &file_name, char separator) {
    // probe dimensions of the csv data
    unsigned int row_count = 0;
    unsigned int column_count = 0;
    std::ifstream file(file_name);
    std::string text_line, text_placeholder;
    std::getline(file, text_line); ++row_count;
    std::stringstream text_line_stream(text_line);
    while(std::getline(text_line_stream, text_placeholder, separator))
        ++column_count;
    while(std::getline(file, text_line))
        ++row_count;
    file.close();
    // use the dimensions to allocate
    // space for the data (with transposition)
    Matrix<2, T>::allocate({column_count, row_count});
}

template<typename T>
T* Data<T>::row(unsigned int row_index) const {
    return this->data + row_index * this->shape[1];
}


unsigned read_single_uint(std::string &&file_name) {
    std::ifstream file(file_name);
    std::string text_line;
    std::getline(file, text_line);
    file.close();
    return static_cast<unsigned int>(std::stoi(text_line));
};

In [4]:
// discrete data
Data<unsigned int> discrete_data_tr;
discrete_data_tr.read_transposing("dummy_discrete_data.csv");
std::cout << "data_tr:" << std::endl;
discrete_data_tr.print(5);

data_tr:
size: 10300000
shape: {10300, 1000}
elements:
0 [0, 0] 3
1 [0, 1] 3
2 [0, 2] 2
...
10299998 [10299, 998] 0
10299999 [10299, 999] 0


In [5]:
// get column_and_bucket_counts 
Data<unsigned int> column_and_bucket_counts_tr;
column_and_bucket_counts_tr.read_transposing("dummy_column_and_bucket_counts.csv");
std::cout << "column_and_bucket_counts_tr:" << std::endl;
column_and_bucket_counts_tr.print()

column_and_bucket_counts_tr:
size: 18
shape: {2, 9}
elements:
4	2	1	117	9886	30	30	30	200	
4	5	6	2	2	4	5	6	2	


In [6]:
// get a tile-width for the cpu workers
unsigned int cpu_tile_width = read_single_uint("dummy_cpu_tile_width.txt");
std::cout << "cpu_tile_width: " << cpu_tile_width;

cpu_tile_width: 13

In [7]:
// Information about contigous regions of columns of (the actual) data are collected in one table.
// We call this table `column_regions`.
// The list of consequtive regions may include only :
//   A) lean interesting columns of a particular dof,
//   B) fat interesting columns
//   C) fat noninteresting (non-contrast) columns
//   D) lean contrast columns of a particular dof (corresponding to A)
//   E) fat contrast columns
// Individual columns of `column_regions` correspond to the regions.
// Rows of `column_regions` are:
//   0. number of columns in the region,
//   1. dof, the number of discrete classes in the data that each column in the region encodes,
//   2. index of the first column in the region,
//   3. one plus index of the last columns in the region,
//   4. number of cpu-tile-widths that fit in the region (possibly including a margin)

// Create the table.
Matrix<2, unsigned int> column_regions;

// Write to rows 0 and 1. by simply copying from the data read earlier from file.
column_regions.allocate({5, column_and_bucket_counts_tr.shape[1]});
for (unsigned int flat_index = 0; flat_index < column_and_bucket_counts_tr.size; ++flat_index) {
    column_regions.data[flat_index] = column_and_bucket_counts_tr.data[flat_index];
}
// Write to rows 2 and 3.
column_regions.elem({3,0}) = column_regions.elem({0,0});
for (unsigned int region_index = 1; region_index < column_regions.shape[1]; ++region_index) {
    column_regions.elem({2, region_index}) = column_regions.elem({3, region_index - 1});
    column_regions.elem({3, region_index}) = column_regions.elem({2, region_index}) + column_regions.elem({0, region_index});
}
// Write to row 4.
for (unsigned int region_index = 0; region_index < column_regions.shape[1]; ++region_index) {
    column_regions.elem({4, region_index}) = (column_regions.elem({0, region_index}) - 1) / cpu_tile_width + 1;
}

In [8]:
std::cout << "column_regions: ";
column_regions.print()

column_regions: size: 45
shape: {5, 9}
elements:
4	2	1	117	9886	30	30	30	200	
4	5	6	2	2	4	5	6	2	
0	4	6	7	124	10010	10040	10070	10100	
4	6	7	124	10010	10040	10070	10100	10300	
1	1	1	9	761	3	3	3	16	


In [9]:
// is_fat_interesting_region_present
bool I_fat_int = read_single_uint("dummy_is_fat_interesting_region_present.txt");
std::cout << "is_fat_interesting_region_present: ";
if (I_fat_int) {
    std::cout << "true";
} else {
    std::cout << "false";
}
std::cout << std::endl;

is_fat_interesting_region_present: true


In [10]:
// the number of lean regions
unsigned int lean_count = read_single_uint("dummy_number_of_lean_regions.txt");
std::cout << "number_of_lean_regions: " << lean_count << std::endl;

number_of_lean_regions: 3


In [11]:
assert (column_regions.shape[1] == 2 * lean_count + 2 + I_fat_int);

In [12]:
class Indices {
   // Abstraction for a finite sequence of indices,
   // where each 'index' is a multi-index i.e. it 
   // is itself a `dim`-sized sequence of integers,
   // say e.g. ((0,0), (0,1), (0,2), (1,1), ... ),
   // for `dim=2`. The integers of a current
   // multi-index are held in a buffer. This class
   // serves to modify the buffer with its `next`
   // method to simulate sequence traversal,
   // i.e. it is a generator.
   public:
      // By the sequence being exhausted we mean
      // that it has been taken beyond its last element.
      // When the index represents the last element of
      // the sequence, the returned value is still false.
      virtual bool is_exhausted() const = 0;
      // Connect some internal pointer to the buffer with indices.
      virtual void use_buff(unsigned int*) = 0;
      // Number of indices (lenght of buffer)
      unsigned int dim;
      // Move the index one step forward in the sequence.
      virtual void next() = 0; 
      // Change the index back to the first element of the sequence.
      virtual void reset() = 0;
      // Return a printable representation of the current index.
      virtual std::string to_str() const = 0;
      // Traverse the sequence, printing subsequent indices.
      void print(unsigned limit = -1) {
         assert(index);
         unsigned count = 0;
         while(!is_exhausted()) {
            if (count == limit)
               break;
            std::cout << to_str() << std::endl;
            next();
            ++count;
         }
         reset();
      }
};

In [13]:
class IndicesTriangle : public Indices {
   // Each index in the sequence is an increasing sequence.
   // Notice that this subclass also implements the
   // degenerated cases of zero- and one-element sequences,
   // as well as the zero-dimensional sequence.
   protected:
      // Pointer to a buffer with actual integers.
      unsigned int* index = NULL;
      // Lower bound (inclusive) of all values of all indices.
      unsigned int lower;
      // Upper bound (noninclusive) of all values of all indices.
      unsigned int upper;
      // Is the index-sequence not strictly increasing;
      // i.e. is e.g. (0, 1, 1, 2) in the sequence?
      bool with_diag;
      // Was the whole sequence exhausted?
      bool exhausted;
   public:
      IndicesTriangle (unsigned int lower_ ,
                       unsigned int upper_,
                       unsigned int dim_,
                       bool with_diag_ = true) :
         lower(lower_),
         upper(upper_),
         with_diag(with_diag_),
         exhausted(lower == upper) {
         dim = dim_;
         assert(upper >= lower);
         assert(with_diag | (upper - lower >= dim));
      }
      bool is_exhausted() const {
         return exhausted;
      }
      void use_buff(unsigned int* const buff) {
         index = buff;
      }
      void next() {
         if (upper == lower) { return ;}
         // F-style traversal
         else if (dim > 0) {
            bool shift_right = 1;
            for(unsigned int position = 0; position < dim - 1; ++position) {
               shift_right = (index[position] + 1) / (index[position + 1] + with_diag);
               if (shift_right) {
                  index[position] = lower + !with_diag * position;
               } else {
                  ++index[position];
                  break;
               }
            }
            if (shift_right) {
               ++index[dim - 1];
               exhausted = index[dim - 1] / upper;
            }
         }
         else {
            exhausted = true;
         }
      }
      void next_() {
         if (upper == lower) { return ;}
         // C-style traversal
         else if (dim > 0) {
            unsigned int position = dim - 1;
            bool shift_left = (index[position] + 1) / upper;
            while (shift_left) {
               if (position == 0) {
                  exhausted = true;
                  return;
               }
               --position;
               shift_left = (index[position] + !with_diag) / index[position + 1];
            }
            ++index[position];
            for (unsigned int i = position + 1; i < dim; ++i) {
               index[i] = index[i - 1] + !with_diag;
            }  
         }
         else {
            exhausted = true;
         }
      }
      void reset() {
         if (upper > lower) {
            for(unsigned int position = 0; position < dim; ++position)
               index[position] = lower + !with_diag * position;
            exhausted = false;
         }
      }
      std::string to_str() const {
         std::stringstream ss;
         if (dim > 0 & upper > lower) {
            ss << "(";
            for (unsigned int i = 0; i < dim - 1; ++i)
                ss << index[i] << ", ";
            ss << index[dim - 1] << ")";
         }
         else {
            ss << "";
         }
         return ss.str();
      }
};

In [14]:
class IndicesProduct : public Indices {   
   // Cartesian product of two sequences.
   // Again results in a sequence (the order
   // is "row-wise", i.e. second index changes
   // faster).
   protected:
      Indices& ind_L;
      Indices& ind_R;
      unsigned int boundary_index;
   public:
      IndicesProduct(Indices& ind_L_arg, Indices& ind_R_arg) :
         ind_L(ind_L_arg),
         ind_R(ind_R_arg),
         boundary_index(ind_L_arg.dim) {
            dim = ind_L_arg.dim + ind_R_arg.dim;
      }
      void use_buff(unsigned int* const buff) {
         ind_L.use_buff(buff);
         ind_R.use_buff(buff + boundary_index);
      }
      bool is_exhausted() const {
         return ind_L.is_exhausted();
      }
      void next() {
         ind_R.next();
         if (ind_R.is_exhausted()) {
            ind_R.reset();
            ind_L.next();
         }
      }
      void reset() {
         ind_L.reset();
         ind_R.reset();
      }
      std::string to_str() const {
         std::stringstream ss;
         ss << ind_L.to_str() << ind_R.to_str();
         return ss.str();
      }
};


class IndicesSum : public Indices {
   // Concatenation of two sequences.
   // (Assumes that both represent multi-indices
   // of the same length `dim`.)
   // Meaning that, when we step over the sum,
   // we first exhaust the first component sequence,
   // and then step through the second one.
   protected:
      Indices& ind_L;
      Indices& ind_R;
      bool using_L;
   public:
      IndicesSum(Indices &ind_L_arg, Indices &ind_R_arg) :
         ind_L(ind_L_arg),
         ind_R(ind_R_arg),
         using_L(true) {
         assert(ind_L_arg.dim == ind_R_arg.dim);
         dim = ind_L_arg.dim;
      }
      void use_buff(unsigned int* const buff) {
         ind_L.use_buff(buff);
         ind_R.use_buff(buff);
      }
      bool is_exhausted() const {
         return ind_L.is_exhausted() & 
                ind_R.is_exhausted();
      }
      void next() {
         if (using_L) {
            ind_L.next();
            if(ind_L.is_exhausted()) {
               using_L = false;
               ind_R.reset();
            }
         }
         else {
            ind_R.next();
         }
      }
      void reset() {
         using_L = true;
         ind_L.reset();
         if(ind_L.is_exhausted()) {
              using_L = false;
              ind_R.reset();
         }
      }
      std::string to_str() const {
         std::stringstream ss;
         ss << (using_L ? ind_L.to_str() : ind_R.to_str());
         return ss.str();
      }
};

In [15]:
template<typename T_in, typename T_cumulation>
T_cumulation& cumulate(std::vector<T_in>& vect,
                       std::vector<T_cumulation>& vect_cum,
                       IndicesTriangle& identity) {
    // Return e.g.
    // \sum_{i = 0}^{n-1} vect[i],
    // where vect is a vector of size n.
    
    assert(vect.size > 0);
    vect_cum.reserve(vect.size());
    vect_cum.emplace_back(identity, vect[0]);
    for(unsigned int i = 1; i < vect.size(); ++i)
        vect_cum.emplace_back(vect_cum.back(), vect[i]);
    
    return vect_cum.back();
}

In [16]:
template<typename T_1, typename T_2>
IndicesSum& dot(std::vector<T_1>& vect_1,
                std::vector<T_2>& vect_2,
                std::vector<IndicesProduct>& vect_1x2,
                std::vector<IndicesSum>& vect_1x2_sum,
                IndicesTriangle& zero) {
    
    // Return
    // \sum_{i = 0}^{n-1} vect_1[i] \times vect_2[i],
    // where vect_1 and vect_2 are vectors of size n.
    
    assert(vect_1.size() == vect_2.size());
    vect_1x2.reserve(vect_1.size());
    
    for(unsigned int i = 0; i < vect_1.size(); ++i)
        vect_1x2.emplace_back(vect_1[i], vect_2[i]);
    
    return cumulate(vect_1x2, vect_1x2_sum, zero);
}

In [17]:
const unsigned int dim = 5;
unsigned int dummy_index[dim];

In [18]:
// \sum_{i=1}^{dim} (0:L)^i \times (L:L+I)^{dim-i}
// where L <- number_of_lean_regions
//       I <- indicator of whether the fat interesting region is present

std::vector<IndicesTriangle> lean;
std::vector<IndicesTriangle> fat_int;
lean.reserve(dim - 1);
fat_int.reserve(dim - 1);
for (unsigned int i = 1; i < dim; ++i) {
    lean.emplace_back(0, lean_count, i);
    fat_int.emplace_back(lean_count, lean_count + I_fat_int, dim - i);
}
std::vector<IndicesProduct> lean_x_fat_int;
std::vector<IndicesSum> lean_x_fat_int_fold;
IndicesTriangle zero(0, 0, dim);
IndicesSum& lean_x_fat_int_sum = dot(lean, fat_int, lean_x_fat_int, lean_x_fat_int_fold, zero); 

In [19]:
// \sum_{i=1}^{dim-1} (0:L)^i \times (L:L+I)^{dim-1-i}
// + (L+I):(2L+2+I)
// where L <- number_of_lean_regions
//       I <- indicator of whether the fat interesting region is present

std::vector<IndicesTriangle> lean_;
std::vector<IndicesTriangle> fat_int_;
lean_.reserve(dim - 2);
fat_int_.reserve(dim - 2);
for (unsigned int i = 1; i < dim - 1; ++i) {
    lean_.emplace_back(0, lean_count, i);
    fat_int_.emplace_back(lean_count, lean_count + I_fat_int, dim - 1 - i);
}
std::vector<IndicesProduct> lean_x_fat_int_;
std::vector<IndicesSum> lean_x_fat_int_fold_;
IndicesTriangle zero_(0, 0, dim - 1);
IndicesSum& lean_x_fat_int_sum_ = dot(lean_, fat_int_, lean_x_fat_int_, lean_x_fat_int_fold_, zero_); 
IndicesTriangle nonint(lean_count + I_fat_int, 2 * lean_count + 2 + I_fat_int, 1);
IndicesProduct lean_x_fat_int_sum_x_nonint(lean_x_fat_int_sum_, nonint);

In [20]:
// (L:L+I)^{dim-1} \times (L+I+1):(2L+I+1)
// where L <- number_of_lean_regions
//       I <- indicator of whether the fat interesting region is present

IndicesTriangle fat_int__(lean_count, lean_count + I_fat_int, dim - 1);
IndicesTriangle lean_contrast(lean_count + I_fat_int + 1,
                              2 * lean_count + I_fat_int + 1, 1);
IndicesProduct fat_int_x_lean_contrast(fat_int__, lean_contrast);

In [21]:
// the not-accelerated task-groups
IndicesSum not_acc_tasks_(lean_x_fat_int_sum, lean_x_fat_int_sum_x_nonint);
IndicesSum not_acc_tasks(not_acc_tasks_, fat_int_x_lean_contrast);

In [22]:
// (L:L+I)^dim
// where L <- number_of_lean_regions
//       I <- indicator of whether the fat interesting region is present

IndicesTriangle fat_int___(lean_count, lean_count + I_fat_int, dim);

// (L:L+I)^{dim-1} \times [(L+I):(L+I+1) + (2L+I+1):(2L+I+2)]
// where L <- number_of_lean_regions
//       I <- indicator of whether the fat interesting region is present

IndicesTriangle fat_nonint_noncontrast(lean_count + I_fat_int, lean_count + I_fat_int + 1, 1);
IndicesTriangle fat_contrast(2 * lean_count + I_fat_int + 1, 2 * lean_count + I_fat_int + 2, 1);
IndicesSum fat_nonint(fat_nonint_noncontrast, fat_contrast);
IndicesProduct fat_int_x_nonint(fat_int[0], fat_nonint);

// the accelerated task-groups
IndicesSum acc_tasks(fat_int___, fat_int_x_nonint);

In [23]:
unsigned int not_acc_task_index[dim];
not_acc_tasks.use_buff(not_acc_task_index);
not_acc_tasks.reset();
not_acc_tasks.print();

(0)(3, 3, 3, 3)
(1)(3, 3, 3, 3)
(2)(3, 3, 3, 3)
(0, 0)(3, 3, 3)
(0, 1)(3, 3, 3)
(1, 1)(3, 3, 3)
(0, 2)(3, 3, 3)
(1, 2)(3, 3, 3)
(2, 2)(3, 3, 3)
(0, 0, 0)(3, 3)
(0, 0, 1)(3, 3)
(0, 1, 1)(3, 3)
(1, 1, 1)(3, 3)
(0, 0, 2)(3, 3)
(0, 1, 2)(3, 3)
(1, 1, 2)(3, 3)
(0, 2, 2)(3, 3)
(1, 2, 2)(3, 3)
(2, 2, 2)(3, 3)
(0, 0, 0, 0)(3)
(0, 0, 0, 1)(3)
(0, 0, 1, 1)(3)
(0, 1, 1, 1)(3)
(1, 1, 1, 1)(3)
(0, 0, 0, 2)(3)
(0, 0, 1, 2)(3)
(0, 1, 1, 2)(3)
(1, 1, 1, 2)(3)
(0, 0, 2, 2)(3)
(0, 1, 2, 2)(3)
(1, 1, 2, 2)(3)
(0, 2, 2, 2)(3)
(1, 2, 2, 2)(3)
(2, 2, 2, 2)(3)
(0)(3, 3, 3)(4)
(0)(3, 3, 3)(5)
(0)(3, 3, 3)(6)
(0)(3, 3, 3)(7)
(0)(3, 3, 3)(8)
(1)(3, 3, 3)(4)
(1)(3, 3, 3)(5)
(1)(3, 3, 3)(6)
(1)(3, 3, 3)(7)
(1)(3, 3, 3)(8)
(2)(3, 3, 3)(4)
(2)(3, 3, 3)(5)
(2)(3, 3, 3)(6)
(2)(3, 3, 3)(7)
(2)(3, 3, 3)(8)
(0, 0)(3, 3)(4)
(0, 0)(3, 3)(5)
(0, 0)(3, 3)(6)
(0, 0)(3, 3)(7)
(0, 0)(3, 3)(8)
(0, 1)(3, 3)(4)
(0, 1)(3, 3)(5)
(0, 1)(3, 3)(6)
(0, 1)(3, 3)(7)
(0, 1)(3, 3)(8)
(1, 1)(3, 3)(4)
(1, 1)(3, 3)(5)
(1, 1)(3, 3)(6)
(1, 1)(3

In [24]:
unsigned int acc_task_index[dim];
acc_tasks.use_buff(acc_task_index);
acc_tasks.reset();
acc_tasks.print();

(3, 3, 3, 3, 3)
(3, 3, 3, 3)(4)
(3, 3, 3, 3)(8)


In [25]:
template<unsigned int dim>
unsigned task_dof(unsigned (&task_index)[dim],
                  const Matrix<2, unsigned> &column_regions) {
    // Map a task-index into product of dofs associated
    // with column-regions encoded by consequtive values
    // in the index. These factor dofs are stored in row 1
    // of the `column_regions` matrix.
    unsigned int dof = 1;
    for (unsigned int dim_index = 0; dim_index < dim; ++dim_index)
        dof *= column_regions.elem({1, task_index[dim_index]});
    return dof;    
}

In [26]:
template<unsigned int dim>
void index_logarithm(const unsigned int (&index)[dim], std::vector<unsigned> &exponents) {
    // Count each succesive integer present in the array.
    // Record this counts in the vector.
    // E.g, map {0,0,2,2,3} into {2,0,2,1,...}.
    // (Elements of the first array have the interpratation of factors in a product:
    // e.g. 0 x 0 x 2 x 2 x 3 = 0^2 x 1^0 x 2^2 x 3^1, hence the name of this function.)
    //
    std::fill(exponents.begin(), exponents.end(), 0);
    for (unsigned int position = 0; position < dim; ++position)
        ++exponents[index[position]];
}

In [27]:
template<unsigned int dim>
void update_column_ranges(Matrix<2, unsigned> &column_ranges,
                          const Matrix<2, unsigned> &column_regions,
                          const unsigned (&task_index)[dim],
                          const unsigned (&subtask_index)[dim],
                          const unsigned tile_width) {
    // Mutate `column-ranges` in-place filling it with starting
    // and ending column-indices which define an atomic query for
    // a worker. These indices are initially encoded by two arrays:
    // the `task_index` and `subtask_index`. The `tile_width`
    // value is used to parcel each column-region (encoded by
    // a single element of the `task_index` array) into column-
    // -ranges of that width. Such parcels are then indexed by
    // elements of the `subtask_index` array.
    //
    for (unsigned int dim_index = 0; dim_index < dim; ++dim_index) {
        unsigned int region_index = task_index[dim_index];
        unsigned int column_index_offset = tile_width * subtask_index[dim_index];
        unsigned int column_index = column_regions.elem({2, region_index}) + column_index_offset;
        column_ranges.elem({dim_index, 0}) = column_index;
        column_ranges.elem({dim_index, 1}) = std::min(column_index + tile_width,
                                                      column_regions.elem({3, region_index}));
    }
}

---

In [29]:
// std::vector<unsigned> not_acc_task_exponents(column_regions.shape[1]);

// std::vector<IndicesTriangle> not_acc_subtask_factors;
// not_acc_subtask_factors.reserve(column_regions.shape[1]);

// std::vector<IndicesProduct> not_acc_subtasks_partial_products;
// IndicesTriangle one(0, 1, 0);

// unsigned int not_acc_subtask_index[dim];

// Matrix<2, unsigned> column_ranges;
// column_ranges.allocate({dim, 2});

// while (!not_acc_tasks.is_exhausted()) {
//     index_logarithm(not_acc_task_index, not_acc_task_exponents);
//     not_acc_subtask_factors = {};
//     for (unsigned int region_index = 0; region_index < column_regions.shape[1]; ++region_index) {
//         not_acc_subtask_factors.emplace_back(0,
//                                              column_regions.elem({4, region_index}),
//                                              not_acc_task_exponents[region_index] );
//     }
//     IndicesProduct& not_acc_subtasks = cumulate(not_acc_subtask_factors,
//                                                 not_acc_subtasks_partial_products,
//                                                 one);
//     not_acc_subtasks.use_buff(not_acc_subtask_index);
//     not_acc_subtasks.reset();
//     while (!not_acc_subtasks.is_exhausted()) {
//         update_column_ranges(column_ranges,
//                              column_regions,
//                              not_acc_task_index,
//                              not_acc_subtask_index,
//                              cpu_tile_width);
//         not_acc_subtasks.next();
//     }    
//     //
//     not_acc_tasks.next();
// }

---

In [30]:
std::vector<unsigned> not_acc_task_exponents(column_regions.shape[1]);

In [31]:
not_acc_task_index

{ 0, 3, 3, 3, 3 }

In [32]:
index_logarithm(not_acc_task_index, not_acc_task_exponents)

In [33]:
not_acc_task_exponents

{ 1, 0, 0, 4, 0, 0, 0, 0, 0 }

In [34]:
std::vector<IndicesTriangle> not_acc_subtask_factors;
not_acc_subtask_factors.reserve(column_regions.shape[1]);

In [35]:
not_acc_subtask_factors = {};
for (unsigned int region_index = 0; region_index < column_regions.shape[1]; ++region_index)
    not_acc_subtask_factors.emplace_back(0, column_regions.elem({4, region_index}), not_acc_task_exponents[region_index]);

In [36]:
std::vector<IndicesProduct> not_acc_subtasks_partial_products;
IndicesTriangle one(0, 1, 0);
IndicesProduct& not_acc_subtasks = cumulate(not_acc_subtask_factors, not_acc_subtasks_partial_products, one);

In [37]:
unsigned int not_acc_subtask_index[dim];
not_acc_subtasks.use_buff(not_acc_subtask_index);

In [38]:
not_acc_subtask_index

{ 0, 0, 0, 0, 0 }

In [39]:
Matrix<2, unsigned> column_ranges;
column_ranges.allocate({dim, 2});

In [40]:
update_column_ranges(column_ranges,
                     column_regions,
                     not_acc_task_index,
                     not_acc_subtask_index,
                     cpu_tile_width);

In [41]:
column_ranges.print();

size: 10
shape: {5, 2}
elements:
0	4	
7	20	
7	20	
7	20	
7	20	


In [42]:
// template<typename T>
// float rho(T *X, T* Y, unsigned int size) {
//     // Pearson's correlation
//     unsigned int Sum_X = 0, Sum_Y = 0;
//     unsigned int Sum_XX = 0, Sum_YY= 0, Sum_XY=0;
//     for (unsigned int index = 0; index < size; ++index) {
//         Sum_X += X[index];
//         Sum_Y += Y[index];
//         Sum_XX += X[index] * X[index];
//         Sum_YY += Y[index] * Y[index];
//         Sum_XY += X[index] * Y[index];
//     }
//     return (float)(size * Sum_XY - Sum_X * Sum_Y) /
//             std::sqrt( (size * Sum_XX - Sum_X * Sum_X) *
//                        (size * Sum_YY - Sum_Y * Sum_Y) );
// }

In [3]:
template<unsigned int dim, typename T>
void marginalize(Matrix<dim, T> &in, Matrix<dim - 1, T> &out, unsigned int reduced_index) {
    // Take `dim`-dimensional matrix `in` and marginalize
    // it along the `reduced_index`'th dimension.
    // Write the result to the `out` matrix.
    //
    assert(reduced_index < dim);
    //
    unsigned int out_shape[dim - 1];
    unsigned int in_dim_index = 0, out_dim_index = 0;
    for (; in_dim_index < dim; ++in_dim_index) {
        if (in_dim_index == reduced_index) { continue; }
        out_shape[out_dim_index++] = in.shape[in_dim_index];
    }
    out.allocate(out_shape);
    //
    unsigned int I = 1;
    for (unsigned int dim_index = 0; dim_index < reduced_index; ++dim_index)
         I *= in.shape[dim_index];
    unsigned int J = in.shape[reduced_index];
    unsigned int K = 1;
    for (unsigned int dim_index = reduced_index + 1; dim_index < dim; ++dim_index)
        K *= in.shape[dim_index];
    unsigned int JK = J * K;
    unsigned int i, j, k;
    for (i = 0; i < I; ++i)
        for (j = 0; j < J; ++j)
            for (k = 0; k < K; ++k)
                out.data[i * K + k] += in.data[i * JK + j * K + k];
}

In [4]:
Matrix<3, int> M;
M.allocate({2, 3, 2});
M.elem({0, 2, 1}) = 13;
M.elem({1, 1, 0}) = 42;
Matrix<2, int> M_1;

In [5]:
M.print()

size: 12
shape: {2, 3, 2}
elements:
0 [0, 0, 0] 0
1 [0, 0, 1] 0
2 [0, 1, 0] 0
3 [0, 1, 1] 0
4 [0, 2, 0] 0
5 [0, 2, 1] 13
6 [1, 0, 0] 0
7 [1, 0, 1] 0
8 [1, 1, 0] 42
9 [1, 1, 1] 0
10 [1, 2, 0] 0
11 [1, 2, 1] 0


In [6]:
marginalize(M, M_1, 1)

In [8]:
M_1.print()

size: 4
shape: {2, 2}
elements:
0	13	
42	0	
